## **Exploring Spotify PlayLists for Track Identification**

In this note book we deploy Soptify API (spotpy) to obtain: 1) all the playlists from a given user id and 2) specific playlist shared by the user

In [129]:
#import spotipy with a developers id and passcode

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="07ca03cba8534f969d080936d5833062"
 
secret = "fbfcbbd68f884d5e887d7b502f97a036"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [130]:
#usual imports

import numpy as np
import scipy.stats
import scipy.special

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm

from bs4 import BeautifulSoup

import pandas as pd
import seaborn as sns

%matplotlib inline

In [131]:
#import prelimenary results
survey = pd.read_csv('survey_final.csv', header=0)

In [132]:
survey=survey.drop([0])
survey=survey.drop([1])
survey.columns


Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q8', 'Q1_1', 'Q1_2', 'Q1_3',
       'Q3_1', 'Q3_2', 'Q3_3', 'Q1_2 - Topics', 'Q3_1 - Topics'],
      dtype='object')

In [188]:
survey.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,UserLanguage,Q8,Q1_1,Q1_2,Q1_3,Q3_1,Q3_2,Q3_3,Q1_2 - Topics,Q3_1 - Topics
2,11/1/18 18:11,11/1/18 18:34,IP Address,71.232.23.32,100,1376,TRUE,11/1/18 18:34,R_3e8NCLhxBzvrO8r,NaN,...,EN,Agree,Erin Williams,erinanthonywilliams@gmail.com,neqe9xtke1qxibqhma4se1vwx,https://open.spotify.com/user/neqe9xtke1qxibqh...,https://open.spotify.com/user/neqe9xtke1qxibqh...,https://open.spotify.com/user/neqe9xtke1qxibqh...,NaN,Unknown
3,11/1/18 19:07,11/1/18 19:14,IP Address,68.34.133.37,100,382,TRUE,11/1/18 19:14,R_R3SlkTHchuvZsL7,NaN,...,EN,Agree,Nadege Benoit,Nbenoit36@gmail.com,Nbenoit36,https://open.spotify.com/user/spotify/playlist...,https://open.spotify.com/user/1221022833/playl...,https://open.spotify.com/user/alena_gorbunova/...,NaN,Unknown
4,11/1/18 19:23,11/1/18 19:26,IP Address,74.196.230.56,100,176,TRUE,11/1/18 19:26,R_1QatFWOKknPGxLj,NaN,...,EN,Agree,Bonnie Huggins,Bonita.a.huggins@gmail.com,Bonnie19belle,https://open.spotify.com/user/bonnie19belle/pl...,https://open.spotify.com/user/bonnie19belle/pl...,https://open.spotify.com/user/digster.fm/playl...,NaN,Unknown
5,11/1/18 19:45,11/1/18 19:51,IP Address,24.21.27.16,100,338,TRUE,11/1/18 19:51,R_1KwgND3pykpGm3D,NaN,...,EN,Agree,Sandra Nippert,Smidjy@aol.com,Sandy Nippert,https://open.spotify.com/user/nickfearless/pla...,https://open.spotify.com/user/giesen.tobi/play...,https://open.spotify.com/user/maranathamusicof...,NaN,Unknown
6,11/1/18 19:48,11/1/18 19:52,IP Address,144.163.26.57,100,262,TRUE,11/1/18 19:52,R_2ATsllX5d8iTw4e,NaN,...,EN,Agree,Sara Bell,Sarakbell2018@gmail.com,skayl6,https://open.spotify.com/user/skayl6/playlist/...,https://open.spotify.com/user/skayl6/playlist/...,https://open.spotify.com/user/skayl6/playlist/...,NaN,Unknown


In [139]:
#get tracks from shared playlists
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    return results

In [140]:
all_users=[]
user_all_playlists=[]
cols=['Q3_1','Q3_2','Q3_3']
i=2
while i <(len(survey)+2):
    username=str(survey.loc[i,['Q1_3']].values[0])
    k=0
    while k<3:
        playlist_link=str(survey.loc[i,[cols[k]]].values[0])
        playlist_id=playlist_link.split('/')[-1]
        playlist=get_playlist_tracks(username,playlist_id) 
        track_uri=[]
        tracks=playlist['tracks']['items']
        m=0
        while m <len(tracks):
            uri=tracks[m]['track']['uri']
            track_uri.append(uri)
            m=m+1
        user_all_playlists.append(track_uri)
        k=k+1
    all_users.append(user_all_playlists)
    i=i+1
print("number of participants:",len(all_users))

number of participants: 42


In [141]:
all_users_tracks=all_users[0]
all_users_tracks=np.array(all_users_tracks).reshape(-1,1)

In [142]:
all_users_tracks=pd.DataFrame(all_users_tracks)
all_users_tracks.columns=["Playlist Tracks"]
all_users_tracks.head()
index_user=[]
vals=list(survey['Q1_1'])
i=0
while i <42:
    n=0
    while n<3:
        m=vals[i]
        index_user.append(m)
        n=n+1
    i=i+1

In [143]:
index_user=pd.DataFrame(index_user)

In [144]:
all_users_tracks=pd.merge(all_users_tracks,index_user, right_index=True, left_index=True)

In [145]:
all_users_tracks.columns=["Playlist Tracks", "User"]

In [187]:
#file with all the track uri's per playlist, per user
all_users_tracks

,Playlist Tracks,User
0,"[spotify:track:0tgVpDi06FyKpA1z0VMD4v, spotify...",Erin Williams
1,"[spotify:track:09IStsImFySgyp0pIQdqAc, spotify...",Erin Williams
2,"[spotify:track:0B4E0IrxBG77W80ZJQTy50, spotify...",Erin Williams
3,"[spotify:track:0bAkKNCQfWkexHFn7fIKns, spotify...",Nadege Benoit
4,"[spotify:track:2YEr07055St40kz0aUfm4f, spotify...",Nadege Benoit
5,"[spotify:track:2FeUnAeRSAJdznrb2lAmrW, spotify...",Nadege Benoit
6,"[spotify:track:3T6MAweakkNrMYs8jZIWtg, spotify...",Bonnie Huggins
7,"[spotify:track:0x3TibRgnSqzkcwrqBd4iX, spotify...",Bonnie Huggins
8,"[spotify:track:6viHvc13l9A6Ua6xugpeX6, spotify...",Bonnie Huggins
9,"[spotify:track:2nOEgBUdiNnfljurqzV1tD, spotify...",Sandra Nippert


In [76]:
# define a user friendly way to display tracks

def show_tracks(results):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print("   %d %32.32s %s" % (i, track['artists'][0]['name'], track['name']))

In [77]:
#get all the tracks from all the playlists that the user has

username=username
playlists = sp.user_playlists(username)
for playlist in playlists['items']:
        if playlist['owner']['id'] == username:
            print()
            print(playlist['name'])   
            print('  total tracks', playlist['tracks']['total'])
            results = sp.user_playlist(username, playlist['id'], fields="tracks,next")
            
            tracks = results['tracks']
            show_tracks(tracks)
            while tracks['next']:
                tracks = sp.next(tracks)
                show_tracks(tracks)



EDC
  total tracks 15
   0                Alison Wonderland I Want U
   1                             Jauz Rock The Party - Radio Edit
   2                          Carnage Bricks
   3                          Carnage I Like Tuh
   4                          Carnage Plur Genocide (feat. Lockdown)
   5                Alison Wonderland Good Enough
   6                            DVBBS Tsunami - Original Mix
   7                         San Holo Light
   8                             Jauz Feel The Volume
   9                         Skrillex Chicken Soup
   10                   Valentino Khan Deep Down Low
   11                    Flosstradamus Soundclash
   12                          NGHTMRE GUD VIBRATIONS
   13                          JOYRYDE I WARE HOUSE
   14                         DJ Snake Propaganda

Driving
  total tracks 13
   0                     Travis Scott SICKO MODE
   1                         Jay Rock WIN
   2                        Sheck Wes Live SheckWes Die SheckWes

In [79]:
tracks1=tracks['items']
tracks1

[{'added_at': '2018-10-31T15:02:19Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/wu8kq8i7s861vw8f5o071slsk'},
   'href': 'https://api.spotify.com/v1/users/wu8kq8i7s861vw8f5o071slsk',
   'id': 'wu8kq8i7s861vw8f5o071slsk',
   'type': 'user',
   'uri': 'spotify:user:wu8kq8i7s861vw8f5o071slsk'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4NJhFmfw43RLBLjQvxDuRS'},
      'href': 'https://api.spotify.com/v1/artists/4NJhFmfw43RLBLjQvxDuRS',
      'id': '4NJhFmfw43RLBLjQvxDuRS',
      'name': 'Wolfgang Amadeus Mozart',
      'type': 'artist',
      'uri': 'spotify:artist:4NJhFmfw43RLBLjQvxDuRS'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/60adCptqwRkANTtVja0bvf'},
      'href': 'https://api.spotify.com/v1/artists/60adCptqwRkANTtVja0bvf',
      'id': '60adCptqwRkANTtVja0bvf',
      'name': 'Academy of Ancient Music

In [174]:
track1_uri=[]
list_of_all_uris=[]
for item in tracks['items']:
    i=0
    while i <len(tracks1):
        uri=tracks1[i]['track']['uri']
        track1_uri.append(uri)
        i=i+1
    list_of_all_uris.append(track1_uri)

In [175]:
list_of_all_uris

[['spotify:track:0B4E0IrxBG77W80ZJQTy50',
  'spotify:track:1O5u9GJaKkZ3EvcIpqGePF',
  'spotify:track:5ryWtuY9LJdqIKQcePENZ3',
  'spotify:track:025C5U3bZaCJmMsYYRpbbY',
  'spotify:track:1SrQkCfmF2ycMtAVpe4h6x',
  'spotify:track:4wZVrrMGdW7J2IPSDRRbRo',
  'spotify:track:6r5IvOcyN2NGNWGLC6AS5O',
  'spotify:track:5HDmqQZ3NYoiQkbrTris5y',
  'spotify:track:1EbklS2pjM17sxCPoofESx',
  'spotify:track:6lTQXJcbOYIg5fYivZou9R',
  'spotify:track:3cT1uCm6weuMywq8c14Awk',
  'spotify:track:6lGiuCuVtBEy9YgqfhhNIi',
  'spotify:track:5Qaxo1bNMJ73vLR058mZpt',
  'spotify:track:5kO3R1fxKjbWXpDU0TAdl4',
  'spotify:track:11natXGVNkPw4TdNG5M8GP',
  'spotify:track:1i1fxkWeaMmKEB4T7zqbzK',
  'spotify:track:0B4E0IrxBG77W80ZJQTy50',
  'spotify:track:1O5u9GJaKkZ3EvcIpqGePF',
  'spotify:track:5ryWtuY9LJdqIKQcePENZ3',
  'spotify:track:025C5U3bZaCJmMsYYRpbbY',
  'spotify:track:1SrQkCfmF2ycMtAVpe4h6x',
  'spotify:track:4wZVrrMGdW7J2IPSDRRbRo',
  'spotify:track:6r5IvOcyN2NGNWGLC6AS5O',
  'spotify:track:5HDmqQZ3NYoiQkbrT